In [1]:
# this code is generated by the Domino Code Assist toolbar button
import domino_code_assist as dca
dca.init()

Cannot show ipywidgets in text

# Palmer penguins

[The Palmer penguins dataset](https://github.com/allisonhorst/palmerpenguins) is a great dataset for data exploration & visualization. 


## Load data
Code Assist includes this dataset, so you can start exploring Code Assist directly.


In [2]:
import domino_code_assist as dca

df_palmer_penguins = dca.data.palmerpenguins()
df_palmer_penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


# Transformations: filter out rows

For our analysis, we like to get rid of some rows that have missing data. Using Code Assist, we can filter out these rows using a UI without having to remember the proper Pandas syntax.

You can edit the transformation by hovering above the next code cell, hover about the blue Code Assist button,  and choosing "Edit".


In [3]:
df_penguins_clean = df_palmer_penguins.loc[df_palmer_penguins["bill_length_mm"].notna()]
df_penguins_clean = df_penguins_clean.loc[df_penguins_clean["sex"].notna()]
df_penguins_clean

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


## Visualizations: exploring penguin bills

To explore the bills of the penguins in this dataset, we create a scatter plot of `bill_length` vs `bill_depth` using the Code Assist UI.

We split the dataset into the two sexes, and three species. Without Code Assist, we would probably have to consult the plotly documentation. Using Code Assist, the options are readily available to us.

We use the "crossfilter" option, which will apply selections made in this plot to be applied as a filter in all other plots and widgets.

You can edit the visualizations by hovering above the next code cell, hover about the blue Code Assist button, and choosing "Edit."


In [4]:
from solara.express import CrossFilteredFigurePlotly
import plotly.express as px

_base_plot_1 = px.scatter(df_penguins_clean, x="bill_length_mm", y="bill_depth_mm", color="species", facet_col="sex")
_base_plot_1.update_layout(dragmode="lasso")
_base_plot_1.update_layout(margin=dict(l=0, r=0, t=40 if _base_plot_1.layout.title.text else 20, b=0))
plot_1 = CrossFilteredFigurePlotly(_base_plot_1)

plot_1

Cannot show ipywidgets in text

# Visualizations: exploring the islands

Do bill lengths depend on the island the penguins live on? To explore this question, we generate a second visualization, where we plot the average `bill_length` per island.

Because we again enabled the "crossfilter" options, we can select one or multiple island by dragging over the histograms. This selection will be applied as a filter in the scatter plot.


You can edit the visualizations by hovering above the next code cell, hover about the blue Code Assist button, and choosing "Edit."


In [5]:
from solara.express import CrossFilteredFigurePlotly
import plotly.express as px

_base_plot_2 = px.histogram(df_penguins_clean, x="island", y="bill_length_mm", histfunc="avg")
_base_plot_2.update_layout(dragmode="select", selectdirection="h")
_base_plot_2.update_layout(margin=dict(l=0, r=0, t=40 if _base_plot_2.layout.title.text else 20, b=0))
plot_2 = CrossFilteredFigurePlotly(_base_plot_2)

plot_2

Cannot show ipywidgets in text

# Crossfilter widgets: selecting the year

To explore any effect or change over the years, we add a widget that allows us to select one or multiple years. This selection is also applied as a filter to the other two visualizations.

You can edit the widget by hovering above the next code cell, hover about the blue Code Assist button, and choosing "Edit".


In [6]:
from solara import CrossFilterSelect

widget_year = CrossFilterSelect(df_penguins_clean, column="year", multiple=True)
widget_year


Cannot show ipywidgets in text

The following two markdown cells will be used in the app we will create.


## Usage

  * Drag in the scatter plot to select a group of penguins. This selection will be applied as a filter to the histogram.
  * Drag to select histograms to select islands. The island selection will be applied as a filter to the scatter plot.
  * Select years to filter the years in the histogram and scatter plot.
  * Double-click a plot to clear the filter.


## Questions

  * Is there an island that only hosts 1 species?
  * Is there a species that is present on all three islands?


# Creating a penguin app

We now combined the visualizations, the widget, and a few markdown cells into an interactive web app. We can lay out out the visual elements on a grid by dragging them around.

With a single click on "Deploy app", we can deploy our app, so we can share it with others.

You can edit the app by hovering above the next code cell, hover about the blue Code Assist button, and choosing "Edit".


In [7]:
from domino_code_assist.deploy import Deployer
import solara


@solara.component
def Page():
    return dca.CardGridLayout([
        {'item': plot_1, 'w': 12, 'h': 15, 'x': 0, 'y': 9},
        {'item': df_penguins_clean, 'w': 3, 'h': 9, 'x': 0, 'y': 27},
        {'item': plot_2, 'w': 9, 'h': 12, 'x': 3, 'y': 24},
        {'item': widget_year, 'w': 3, 'h': 3, 'x': 0, 'y': 24},
        {'item': dca.MarkdownFromCell('1d9ec016-53b4-4d93-9516-3b32595be313'), 'w': 12, 'h': 4, 'x': 0, 'y': 0},
        {'item': dca.MarkdownFromCell('2bedca56-2690-4a48-b4ca-43052110f2d8'), 'w': 6, 'h': 5, 'x': 0, 'y': 4},
        {'item': dca.MarkdownFromCell('2b556ffa-0058-4065-ad4c-0bd74b02c830'), 'w': 6, 'h': 5, 'x': 6, 'y': 4}
    ])

Deployer(Page)

Cannot show ipywidgets in text